In [1]:
import numpy as np
import scipy
from scipy.linalg import expm
import random
import math
from scipy.stats import unitary_group
import matplotlib.pyplot as plt
from scipy.optimize import leastsq

In [9]:
def create_un_mat(v):
    Mc = np.zeros((4, 4),dtype=np.complex128)
    Mc[0][1] = v[0] + 1j * v[1]
    Mc[1][0] = v[0] - 1j * v[1]
    
    Mc[0][2] = v[2] + 1j * v[3]
    Mc[2][0] = v[2] - 1j * v[3]
    
    Mc[0][3] = v[4] + 1j * v[5]
    Mc[3][0] = v[4] - 1j * v[5]

    Mc[1][2] = v[6] + 1j * v[7]
    Mc[2][1] = v[6] - 1j * v[7]

    Mc[1][3] = v[8] + 1j * v[9]
    Mc[3][1] = v[8] - 1j * v[9]

    Mc[2][3] = v[10] + 1j * v[11]
    Mc[3][2] = v[10] - 1j * v[11]
    
    U = expm(1j * Mc)
    
    return U



def f(v, data):
    M1 = create_un_mat(v[:12])
    M2 = create_un_mat(v[12:24])
    alpha = np.reshape(v[24:33]*0.001, (3, 3))
    h_0 = v[33:36]
    cost = 0
    for i in data:
        x = np.zeros(3)
        x[i[0]] = i[2] ** 2
        y = i[3:]
        y /= np.sum(y)
        h_list = h_0 + alpha.dot(np.transpose(x))
        H = np.diag([math.e ** (1j * h) for h in h_list] + [1])
        predict = (np.abs(np.transpose(M2.dot(H).dot(M1)))** 2)[i[1]]
        cost += np.linalg.norm(np.array(predict)[0] - y)
    return cost

def residuals(v, data):
    M1 = create_un_mat(v[:12])
    M2 = create_un_mat(v[12:24])
    alpha = np.reshape(v[24:33], (3, 3))
    h_0 = v[33:36]
    power = v[36]
    cost = []
    for i in data:
        x = np.zeros(3)
        x[i[0]] = i[2] ** power
        y = i[3:]
        y /= np.sum(y)
        h_list = h_0 + alpha.dot(np.transpose(x))
        H = np.diag([math.e ** (1j * h) for h in h_list] + [1])
        predict = (np.abs(np.transpose(M2.dot(H).dot(M1)))** 2)[i[1]]
        cost.append(np.linalg.norm(np.array(predict) - y))
    return cost


def numerical_gradient(data, x, epsilon=1e-8):
    grad = np.zeros_like(x)
    f_0 = f(x, data)
    for i in range(len(x)):
        x_up = np.copy(x)
        x_down = np.copy(x)
        x_up[i] += epsilon
        x_down[i] -= epsilon
        grad[i] = (f_0 - f(x_down, data)) / (2 * epsilon)
    return np.sign(grad)


def gradient_descent(data, x0, learning_rate_start=0.01, learning_rate_finish=0.01, max_iter=1000, tol=1e-4):
    x = np.array(x0, dtype=float)
    for i in range(max_iter):
        learning_rate = (1 - i / max_iter) * learning_rate_start + learning_rate_finish
        grad = numerical_gradient(data, x)
        x_new = np.ones(37)
        for num, val in enumerate(x0):
            if num in range(24,33):
                x_new[num] = x[num] - learning_rate * grad[num]*10**(-4)
            else:
                x_new[num] = x[num] - learning_rate * grad[num]
            
        if i % 10 == 0:
            print(x_new)
            print("Iteration = ", i, "difference = ", np.linalg.norm(x_new - x))
            print(f(x, data))
        if f(x_new, data) - f(x, data) < tol:
            print(f"Сходимость достигнута за {i+1} итераций.")
            break
        x = x_new
    return x, f(x, data)

# def output_func(x_data, *args):
#     M1 = create_un_mat(args[:12])
#     M2 = create_un_mat(args[12:24])
#     alpha = np.reshape(args[24:33], (3, 3))
#     h_0 = args[33:36]
#     power = args[36]
#     cost = 0
#     i = x_data
#     x[i[0]]=1000000 * i[2] ** 2
#     y = i[3:]
#     y /= np.sum(y)
#     h_list = h_0 + alpha.dot(np.transpose(x))
#     H = np.diag([math.e ** (1j * h) for h in h_list] + [1])
#     predict = (np.abs(np.transpose(M2.dot(H).dot(M1)))** 2)[i[1]]
    # return (np.abs(np.transpose(M2.dot(H).dot(M1)))** 2)[i[1]]

In [ ]:
data = []
for ch in range(4):
    for H in range(3):
        with open(f"ch{ch + 1}_H{H + 9}.txt") as file:
            for line in file:
                try:
                    row = [float(num) for num in line.split()]
                    row[0] *=1000
                    data.append([H, ch] + row)
                except:
                    pass

x0 = np.random.random(37)
x0 [24:33] = 5.5*10**(-4)
x0 [33:36] = -0.385
x0 [36] = 2.567
# x0 = [-3.32853195e+00, -1.06504139e+01, -8.61838138e+00, -4.29003714e+00,
#   3.67028142e+00,  4.61936822e+00, -5.86491425e+00,  1.15434587e+01,
#   3.15385040e+00,  1.10929040e+01, -8.08086041e-01, -2.61565947e+00,
#   9.28736156e+00,  2.15937593e+00,  1.87274358e+00,  3.64234625e+00,
#  -7.27976747e+00, -3.60924993e+00,  2.00773060e+00, -6.91896656e-01,
#  -8.16046681e-01,  1.51303364e+01, -4.43734301e+00,  1.05119799e+01,
#  -9.04806571e-05, -9.93863787e-05, -9.16415297e-05, -1.15026917e-04,
#  -1.19259602e-04, -1.07322522e-04, -9.05611621e-05, -1.00036243e-04,
#  -9.16931873e-05,  1.86719083e-01,  8.34968420e-01, -2.50598954e+00,
#   3.20710143e+00]


# print(x0)
minimum_x, minimum_f = gradient_descent(data, x0, learning_rate_start=0.1, learning_rate_finish=0.005, max_iter=1000)
# print(f"Минимум функции: f({minimum_x}) = {minimum_f}")
# minimum_x, _ = leastsq(residuals, x0, args=data, maxfev = 150000)

In [ ]:
print(minimum_x)

In [1]:
params = minimum_x
# params = np.random.random(37)*10
params [24:33] = 5.5*10**(-5)
params [33:36] = -0.385
params [36] = 2.567
# print(minimum_x)

M1 = create_un_mat(params[:12])
M2 = create_un_mat(params[12:24])
alpha = np.reshape(params[24:33], (3, 3))
h_0 = params[33:36]
power = params[36]

print(M1, M2, alpha, h_0)

for i in data[0:84]:
    x = np.zeros(3)
    x[i[0]] = i[2] ** power
    h_list = h_0 + alpha.dot(np.transpose(x))
    H = np.diag([math.e ** (1j * h) for h in h_list] + [1])
    result = (np.abs(np.transpose(M2.dot(H).dot(M1))) ** 2)[i[1]]
    plt.scatter(i[2], result[0], color = 'blue')
    plt.scatter(i[2], result[1], color = 'green')
    plt.scatter(i[2], result[2], color = 'yellow')
    plt.scatter(i[2], result[3], color = 'red')
    # plt.scatter(1000*i[2], result[0]+result[1]+result[2]+result[3], color = 'black')

plt.plot([d[2] for d in data[0:84]], [d[3]/sum(d[3:]) for d in data[0:84]])#, color = 'blue')
plt.plot([d[2] for d in data[0:84]], [d[4]/sum(d[3:]) for d in data[0:84]])#, color = 'green')
plt.plot([d[2] for d in data[0:84]], [d[5]/sum(d[3:]) for d in data[0:84]])#, color = 'yellow')
plt.plot([d[2] for d in data[0:84]], [d[6]/sum(d[3:]) for d in data[0:84]])#, color = 'red')
plt.plot([d[2] for d in data[0:84]], [d[6]/sum(d[3:]) for d in data[0:84]])



NameError: name 'minimum_x' is not defined